In [1]:
%pwd

u'/home/bks4line/Documents/LTA/Analysis'

In [2]:
%cd Test

/home/bks4line/Documents/LTA/Analysis/Test


In [3]:
path = %pwd

In [4]:
import pandas as pd

In [5]:
import glob
all_files =  glob.glob(path+'/*')

In [6]:
all_files.sort()

In [8]:
len(all_files)

172

In [9]:
prev_time_stamp = all_files[0][48:67] 
#  57:76

In [10]:
prev_time_stamp

u'2017-03-14_01:54:09'

In [11]:
prev_date = pd.to_datetime(prev_time_stamp,format='%Y-%m-%d_%H:%M:%S').day

In [12]:
current_day_df =  pd.DataFrame()

In [13]:
for i in all_files:
    curr_time_stamp = i[48:67]
    curr_date = pd.to_datetime(curr_time_stamp,format='%Y-%m-%d_%H:%M:%S').day
#     if prev day same as current date. concatenate to existing dataframe
    current_csv = pd.read_csv(i)
    current_csv.date = pd.to_datetime(current_csv.date,format='%Y-%m-%d',errors='coerce')
    current_csv = current_csv[~current_csv.date.isnull()]
    
    if curr_date==prev_date:
#         check for unique dates
        unique_dates = list(current_csv.date.dt.day.unique())
        if len(unique_dates)!=1:
            current_day_csv = current_csv[current_csv.date.dt.day == unique_dates[0]]
            next_day_csv = current_csv[current_csv.date.dt.day == unique_dates[1]]
            current_day_df = pd.concat([current_day_df,current_day_csv])
        else:
            current_day_df = pd.concat([current_day_df,current_csv])
            
#     we should have 2 dfs here
#  current_day and next_day
    else:
        print curr_date,prev_date
#         check if prev day is there in current_csv
        yesterday_df = current_csv[current_csv.date.dt.day==prev_date]
        if not yesterday_df.empty:
            current_day_df = pd.concat([current_day_df,yesterday_df])
        file_name =  '/home/bks4line/Documents/LTA/Analysis/output/taxi_report_'+str(prev_date)+'.csv'
        current_day_df.to_csv(file_name)
        if not next_day_csv.empty:
            current_day_df = pd.DataFrame()
            current_day_df = pd.concat([current_day_df,next_day_csv])
            current_day_df = pd.concat([current_day_df,current_csv[current_csv.date.dt.day==curr_date]])
        else:
            current_day_df = current_csv[current_csv.date.dt.day==curr_date]
        prev_date = curr_date

/home/bks4line/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/bks4line/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/bks4line/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


15 14
16 15
17 16
18 17
19 18


In [14]:
file_name =  '/home/bks4line/Documents/LTA/Analysis/output/taxi_report_'+str(prev_date)+'.csv'
current_day_df.to_csv(file_name)

In [ ]:
len(current_day_df)

In [ ]:
len('/home/bks4line/Documents/LTA/Analysis/taxi_data_viz/taxi_2017-03-14_01:54:09.csv')

In [ ]:
def group_df_by_hours(pandas_df):
    
#     1. change the index to time stamp
#     2. group by hours
#     3. return grouped df
#     pandas_df.time = pd.to_datetime(pandas_df.time)
    pandas_df['hour'] = pd.to_datetime(pandas_df['time'], format='%H:%M:%S', errors='coerce').dt.hour
    
    hour_count = pd.DataFrame({'taxi_api_count' : pandas_df.groupby(pandas_df.hour).size()}).reset_index()
    return hour_count

In [ ]:
#  read each file, change index to timestamp, group by column and create to new dataframe.
#  append


hourly_grouped_df =  pd.DataFrame()
for file in all_files:
    if hourly_grouped_df.empty:
        hourly_grouped_df = group_df_by_hours(pd.read_csv(file))
    else:
#         print "not empty "
        hourly_grouped_df = pd.concat([hourly_grouped_df,group_df_by_hours(pd.read_csv(file))])
    
    

In [ ]:
hourly_grouped_df.head()

In [ ]:
hourly_grouped_df = hourly_grouped_df.sort(['hour','taxi_api_count'], ascending=[1,1])

In [ ]:
hourly_grouped_df.head()

In [ ]:
final_list = []
for i in range(24):
    sample = hourly_grouped_df[hourly_grouped_df['hour']==i]['taxi_api_count']
    final_list.append([sample.sum(),sample.std()])

In [ ]:
means,stds = zip(*final_list)

In [ ]:
c

In [ ]:
import plotly.plotly as py
import plotly.graph_objs as go


# def create_trace(list_group, list_group_id):
#     trace = go.Box(
#         y=list_group,
#         name = str(list_group_id),
#         marker = dict(
#         color = 'rgb(10, 140, 208)'))
#     return trace

In [ ]:
# trace_list = []
# for index,value in enumerate(final_list):
#     trace_list.append(create_trace(value,index))

# py.iplot(trace_list)

In [ ]:
data = [
    go.Scatter(
        x=[i for i in range(24)],
        y=means,
        error_y=dict(
            type='data',
            array=stds,
            visible=True
        )
    )
]

In [ ]:
py.iplot(data)

In [ ]:
import matplotlib.pyplot
%matplotlib inline
hourly_grouped_df.groupby(hourly_grouped_df.hour).mean().plot()